In [50]:
#Importing the necessary libraries
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import UnexpectedAlertPresentException, NoAlertPresentException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains

In [51]:
# Initializing the Google WebDriver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [52]:
# Searching the Website on Google
url = "https://www.otcmarkets.com/"
driver.get(url)

In [132]:
# Starting to Handle the Page now
Accept_Cookies = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[2]/div[1]/div/div/div[2]/button")
Accept_Cookies.click()

In [53]:
Stock_Screener = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[1]/div[2]/a")
Stock_Screener.click()

In [54]:
time.sleep(5)

# Getting the current window handle
original_window = driver.current_window_handle

# Getting all window handles
all_windows = driver.window_handles

# Switching to the new window (assuming it is the second window)
for window in all_windows:
    if window != original_window:
        driver.switch_to.window(window)
        break

Second_Window = driver.current_window_handle
rows = driver.find_elements(By.XPATH, '/html/body/div/div/div[4]/div/div[1]/table/tbody/tr')
j = 0

In [55]:
# Column names for Csv File
Company_Data = {
    "Company Symbol": [],
    "Market Cap": [],
    "Authorized Shares": [],
    "Outstanding Shares": [],
    "Par Value": [],
    "SEC Filings": [],
    "Address":[],
    "Website": [],
    "Phone": [],
    "Email": [],
    "Audited Financials": [],
    "Latest Report": [],
    "Fiscal Year End" : [],
    "Company Officers": [],
    "BOARD OF DIRECTORS": [],
    "Employees": []
}

In [56]:
for i in range(0,75):
    counter = 0
    for i in range(len(rows)):
        counter += 1
        print(f"Total Rows: {len(rows)}")
        # Constructing the XPath for the link in the first column of the current row
        xpath = f'/html/body/div/div/div[4]/div/div[1]/table/tbody/tr[{i+1}]/td[1]/a'

        # Finding the link using the constructed XPath
        link = driver.find_element(By.XPATH, xpath)

        # Clicking the link
        link.click()
        time.sleep(3)

        # Getting all window handles again
        all_windows = driver.window_handles

        # Switching to the new window (assuming it is the third window)

        for window in all_windows:
            if window != original_window and window != Second_Window:
                driver.switch_to.window(window)
                break

           # Extracting data from the new page
        try:
            time.sleep(2)
            company_symbol = driver.find_element(By.CLASS_NAME, 'CompanyQuotePage__companySymbol___2D4Xg').text
        except UnexpectedAlertPresentException:
            while True:
                try:
                    alert = driver.switch_to.alert
                    alert.accept()
                    time.sleep(2)  # Waiting before checking again
                except NoAlertPresentException:
                    break
            company_symbol = driver.find_element(By.CLASS_NAME, 'CompanyQuotePage__companySymbol___2D4Xg').text

        time.sleep(1)  # Giving a slight delay

        try:
            driver.execute_script("window.scrollBy(0, 300);")
            time.sleep(2)
        except UnexpectedAlertPresentException:
            while True:
                try:
                    alert = driver.switch_to.alert
                    alert.accept()
                    time.sleep(2)  # Waiting before checking again
                except NoAlertPresentException:
                    break
            driver.execute_script("window.scrollBy(0, 300);")

        time.sleep(2)  # Waiting for the page to load

        try:
            security_details = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[2]/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div/div[5]/a')
        except UnexpectedAlertPresentException:
            while True:
                try:
                    alert = driver.switch_to.alert
                    alert.accept()
                    time.sleep(2)  # Waiting before checking again
                except NoAlertPresentException:
                    break

        security_details = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[2]/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div/div[5]/a')
        security_details.click()
        driver.execute_script("window.scrollBy(0, 600);")
        time.sleep(2)  # Waiting for the page to load

        market_cap = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[2]/div[2]/div[2]/div/div[1]/div[2]/div/div[2]/div/div[2]/div[2]').text
        authorized_shares = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[2]/div[2]/div[2]/div/div[1]/div[2]/div/div[2]/div/div[3]/div[2]').text
        outstanding_shares = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[2]/div[2]/div[2]/div/div[1]/div[2]/div/div[2]/div/div[4]/div[2]').text
        par_value = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[2]/div[2]/div[2]/div/div[1]/div[2]/div/div[2]/div/div[9]/div[2]').text

        if market_cap == "Not Available":
            market_cap = "NaN"
        if authorized_shares == "Not Available":
            authorized_shares = "NaN"
        if outstanding_shares == "Not Available":
            outstanding_shares = "NaN" 
        if par_value == "Not Available" or par_value == "No Par Value":
            par_value = "NaN"

        print(market_cap)
        print(authorized_shares)
        print(outstanding_shares)
        print(par_value)

        driver.execute_script("window.scrollBy(0, -300);")
        time.sleep(2)  # Waiting for the page to load

        Disclosure = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[2]/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div/div[8]/a')
        Disclosure.click()

        driver.execute_script("window.scrollBy(0, 900);")
        time.sleep(2)  # Waiting for the page to load

        sec_filings = "NaN"
        try:
            # Attempt to find and retrieve the element
            sec_filings = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[2]/div[2]/div[2]/div/div[1]/div[2]/div/div[2]/div/div[5]/div/div[1]/table/tbody/tr[1]/td[3]/span/span').text
        except NoSuchElementException:
            # If the element is not found, filling with "Nan"
            sec_filings = "NaN"

        print(sec_filings)

        time.sleep(2)
        driver.execute_script("window.scrollBy(0, -600);")
        time.sleep(2)  # Waiting for the page to load

        Company_Profile = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[2]/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div/div[4]/a')
        Company_Profile.click()

        driver.execute_script("window.scrollBy(0, 600);")
        time.sleep(2)  # Waiting for the page to load

        # Getting Address
        # Finding the main div containing the company info
        main_div = driver.find_element(By.CLASS_NAME, "CompanyQuoteProfile__companyInfo___2zvyM")

        # Finding all <p> tags within the main div, excluding those after the specified div
        address_p_tags = main_div.find_elements(By.XPATH, "./p[following-sibling::div[contains(@class, 'CompanyQuoteProfile__socialButtons___2Ch7j')]]")

        # Initializing an empty list to store the address lines
        address_lines = []

        # Extracting the text from each <p> tag and append it to the address_lines list
        for p_tag in address_p_tags:
            address_lines.append(p_tag.text)

        # Joining the address lines into a single string
        address = " ".join(address_lines)
        if not address:
            address = "NaN"

        print(address)

        # Locating the sub div containing website, phone number, and email
        try:
            contact_div = main_div.find_element(By.XPATH, ".//div[@class='styles__default___3EC_N styles__outline___35742 styles__section2___3xacj']")
        except NoSuchElementException:
            contact_div = None

        # Extracting the website, phone number, and email
        if contact_div:
            try:
                website = contact_div.find_element(By.XPATH, ".//a[1]").text
            except NoSuchElementException:
                website = "NaN"

            try:
                phone = contact_div.find_element(By.XPATH, ".//p").text
            except NoSuchElementException:
                phone  = "NaN"

            try:
                email = contact_div.find_element(By.XPATH, ".//a[2]").text
            except NoSuchElementException:
                email = "NaN"
        else:
            website = phone = email = "NaN"

        print(website)
        print(phone)
        print(email)

        # Initializing variables to store the extracted texts
        audited_financials, latest_report, fiscal_year_end = "NaN", "NaN", "NaN"

        # Loop through the indices
        main_div = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[2]/div[2]/div[2]/div/div[1]/div[2]/div/div[2]/div/div[2]/div[3]')

        # Locating the sub divs within the main div
        sub_divs = main_div.find_elements(By.CLASS_NAME, "sc-htpNat.jtWIOA.sc-bdVaJa.gRrvFh")

        for sub_div in sub_divs:
            # Locating the first sub div by class name
            first_div = sub_div.find_element(By.CLASS_NAME, "sc-bdVaJa.jaTTAC")
            # Locating the second sub div by class name
            second_div = sub_div.find_element(By.CLASS_NAME, "styles__firstColumn___383TU.sc-bdVaJa.iHZvIS")

            # Extracting text from both divs
            if first_div.text == "Audited Financials":
                if second_div.text == "Not Available":
                    audited_financials = "NaN"
                else:
                    audited_financials = second_div.text
            if first_div.text == "Latest Report":
                if second_div.text == "Not Available":
                    latest_report = "NaN"
                else:
                    latest_report = second_div.text
            if first_div.text == "Fiscal Year End":
                if second_div.text == "Not Available":
                    fiscal_year_end = "NaN"
                else:
                    fiscal_year_end = second_div.text

        # Printing the texts in the desired order

        print (audited_financials)
        print (latest_report)
        print (fiscal_year_end)

        main_div_xpath = '//*[@id="root"]/div/div[2]/div[2]/div[2]/div[2]/div/div[1]/div[2]/div/div[2]/div/div[3]'
        main_div = driver.find_element(By.XPATH, main_div_xpath)

        # Finding all relevant child divs within the main div
        child_divs = main_div.find_elements(By.CLASS_NAME, "styles__item___B9e-1.styles__compact___36meA")

        # Listing to store the extracted text pairs
        extracted_data = []

        # Iterating through the child divs
        for child_div in child_divs:
            try:
                # Getting the text from the first div (with class "sc-bdVaJa jaTTAC")
                first_div_text = child_div.find_element(By.CLASS_NAME, "sc-bdVaJa.jaTTAC").find_element(By.TAG_NAME, "b").text

                # Getting the text from the second div (with class "styles__firstColumn___383TU sc-bdVaJa iHZvIS")
                second_div_text = child_div.find_element(By.CLASS_NAME, "styles__item___12taM.sc-htpNat.jtWIOA.sc-bdVaJa.ensNgD").find_element(By.CLASS_NAME, "styles__firstColumn___383TU.sc-bdVaJa.iHZvIS").text

                # Appending the text pair to the list
                extracted_data.append(f"{first_div_text} ({second_div_text})")
            except Exception:
                continue

        # Joining all texts with commas
        company_officers = ', '.join(extracted_data)
        if company_officers == "":
            company_officers = "NaN"

        # Printing or use the company_officers variable
        print(company_officers)


        main_div_xpath1 = '//*[@id="root"]/div/div[2]/div[2]/div[2]/div[2]/div/div[1]/div[2]/div/div[2]/div/div[4]/div[3]'
        main_div1 = driver.find_element(By.XPATH, main_div_xpath1)

        # Finding all relevant child divs within the main div
        child_divs1 = main_div1.find_elements(By.CLASS_NAME, "styles__item___B9e-1.styles__compact___36meA")

        # Initializing an empty list to store the text
        texts = []

        # Iterating through each child div to find the sub div elements and get their text
        for child_div1 in child_divs1:
            try:
                # Getting the text from the first div (with class "sc-bdVaJa jaTTAC")
                first_div_text1 = child_div1.find_element(By.CLASS_NAME, "sc-bdVaJa.jaTTAC").find_element(By.TAG_NAME, "b").text

                # Appending the text pair to the list
                texts.append(first_div_text1)
            except Exception:
                continue

        # Joining all texts with commas
        board_of_directors = ', '.join(texts)
        if board_of_directors == "":
            board_of_directors = "NaN"
        print(board_of_directors)

        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(2)  # Waiting for the page to load
        try:
            employees = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[2]/div[2]/div[2]/div/div[1]/div[2]/div/div[2]/div/div[7]/div[3]/div[3]/div[1]/div/div[2]/div').text
            if employees == "Not Available":
                employees = "NaN"
        except:
            employees = "Nan"
        print(employees)
        # Appending the data to the dictionary   

        Company_Data["Company Symbol"].append(company_symbol)
        Company_Data["Market Cap"].append(market_cap)
        Company_Data["Authorized Shares"].append(authorized_shares)
        Company_Data["Outstanding Shares"].append(outstanding_shares)
        Company_Data["Par Value"].append(par_value)
        Company_Data["SEC Filings"].append(sec_filings)
        Company_Data["Address"].append(address)
        Company_Data["Website"].append(website)
        Company_Data["Phone"].append(phone)
        Company_Data["Email"].append(email)
        Company_Data["Audited Financials"].append(audited_financials)
        Company_Data["Latest Report"].append(latest_report)
        Company_Data["Fiscal Year End"].append(fiscal_year_end)
        Company_Data["Company Officers"].append(company_officers)
        Company_Data["BOARD OF DIRECTORS"].append(board_of_directors)
        Company_Data["Employees"].append(employees)

        # Closing the WebDriver
        driver.close()
        driver.switch_to.window(Second_Window)
        if j == 350:
            break
        time.sleep(5)
        print(f"Value of counter : {counter}")
        if counter == 20:
            counter = 0
            j += 1
            try:
                driver.find_element(By.XPATH, "/html/body/div/div/div[4]/div/div[2]/div[2]/div/ul/li[12]/a").click()

            except NoSuchWindowException:
                 driver.find_element(By.XPATH, "/html/body/div/div/div[4]/div/div[2]/div[2]/div/ul/li[12]/a/span").click()

            time.sleep(5)
            driver.execute_script("window.scrollBy(0, -1100);")
            time.sleep(2)  # Waiting for the page to load
            rows = driver.find_elements(By.XPATH, '/html/body/div/div/div[4]/div/div[1]/table/tbody/tr')


Total Rows: 20
13,847,305
NaN
110,955,967
NaN
NaN
510-1100 Melville Street Vancouver, BC V6E 4A6 Canada
www.strategicmetalsltd.com
+1 604 687 2522
rdrechsler@strategicmetalsltd.com
Audited
NaN
12/31
NaN
NaN
NaN
Value of counter : 1
Total Rows: 20
3,046,798,248
NaN
6,009,464,000
NaN
NaN
19th Floor, Wisma Sime Darby Jalan Raja Laut Kuala Lumpur 50350 Malaysia
www.simedarby.com
(603) 2691 4122
investor.relations@simedarby.com
Audited
NaN
6/30
Dato' Mohd Bakke Salleh (President, CEO), Tong Poh Keow (CFO), Benjamin Poh Chee Fong (VP, Investor Rel.)
NaN
NaN
Value of counter : 2
Total Rows: 20
33,944,976,768
NaN
71,869,360
NaN

Chiyoda-ku 4-14-1 Soto-Kanda Tokyo 101-0021 Japan
www.smcworld.com
+81 3-5207-8271
NaN
Audited
NaN
3/31
Katsunori Maruyama (President)
NaN
15,706 as of 09/30/2012
Value of counter : 3
Total Rows: 20
18,540,894,554
NaN
714,747,000
NaN
NaN
Otto-Hahn-Ring 6 Munchen 81739 Germany
www.siemens-energy.com/global/en.html
+49 (89) 78050
contact@siemens.com
NaN
NaN
9/30
NaN
NaN


Audited
NaN
12/31
Tonny Surya Kusnadi (President)
NaN
479 as of 03/31/2013
Value of counter : 14
Total Rows: 20
6,260,364
Unlimited
189,421,001
NaN
NaN
300 - 2489 Bellevue Avenue West Vancouver, BC V7V 1E1 Canada
www.sonorogold.com
604-632-1764
info@sonorogold.com
Audited
03/31/2024 Quarterly Report
12/31
Kenneth MacLeod (President, CEO), Jorge Diaz (VP, Ops.), Salil Dhaumya (CFO), Katharine Elizabeth Regan (Corporate Secretary, VP, Corp. Dev.), Melvin Herdrick (VP Expl.)
John Michael Darch, Jorge Diaz, Stephen Kenwood, Kenneth MacLeod, Katharine Elizabeth Regan, James Taylor, Curtis Turner
14 as of 12/31/2022
Value of counter : 15
Total Rows: 20
173,426,234
NaN
93,743,910
NaN
NaN
Ryensvingen 7 Oslo 0680 Norway
www.smartoptics.com
+46 704 264 872
mikael.haag@smartoptics.com
NaN
NaN
12/31
NaN
NaN
NaN
Value of counter : 16
Total Rows: 20
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
Value of counter : 17
Total Rows: 20
2,703,912
Unlimited
67,597,800
NaN
NaN
420 - 789 West P

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=124.0.6367.60)
Stacktrace:
#0 0x5a388a15edc3 <unknown>
#1 0x5a3889e4d4e7 <unknown>
#2 0x5a3889e32d95 <unknown>
#3 0x5a3889e336ef <unknown>
#4 0x5a3889e4da40 <unknown>
#5 0x5a3889e236be <unknown>
#6 0x5a3889eca0fa <unknown>
#7 0x5a3889eba418 <unknown>
#8 0x5a3889e8b28a <unknown>
#9 0x5a3889e8bc5e <unknown>
#10 0x5a388a1230eb <unknown>
#11 0x5a388a12703b <unknown>
#12 0x5a388a10f201 <unknown>
#13 0x5a388a127ba2 <unknown>
#14 0x5a388a0f40bf <unknown>
#15 0x5a388a14df18 <unknown>
#16 0x5a388a14e0f0 <unknown>
#17 0x5a388a15df14 <unknown>
#18 0x7e8d8b694ac3 <unknown>


In [57]:
# Making the DataFrame
df = pd.DataFrame(Company_Data)
df

,Company Symbol,Market Cap,Authorized Shares,Outstanding Shares,Par Value,SEC Filings,Address,Website,Phone,Email,Audited Financials,Latest Report,Fiscal Year End,Company Officers,BOARD OF DIRECTORS,Employees
0,SMDZF,"13,847,305",NaN,"110,955,967",NaN,NaN,"510-1100 Melville Street Vancouver, BC V6E 4A6...",www.strategicmetalsltd.com,+1 604 687 2522,rdrechsler@strategicmetalsltd.com,Audited,NaN,12/31,NaN,NaN,NaN
1,SMEBF,"3,046,798,248",NaN,"6,009,464,000",NaN,NaN,"19th Floor, Wisma Sime Darby Jalan Raja Laut K...",www.simedarby.com,(603) 2691 4122,investor.relations@simedarby.com,Audited,NaN,6/30,"Dato' Mohd Bakke Salleh (President, CEO), Tong...",NaN,NaN
2,SMECF,"33,944,976,768",NaN,"71,869,360",NaN,,Chiyoda-ku 4-14-1 Soto-Kanda Tokyo 101-0021 Japan,www.smcworld.com,+81 3-5207-8271,NaN,Audited,NaN,3/31,Katsunori Maruyama (President),NaN,"15,706 as of 09/30/2012"
3,SMEGF,"18,540,894,554",NaN,"714,747,000",NaN,NaN,Otto-Hahn-Ring 6 Munchen 81739 Germany,www.siemens-energy.com/global/en.html,+49 (89) 78050,contact@siemens.com,NaN,NaN,9/30,NaN,NaN,NaN
4,SMEV,"4,206,967","1,800,000,000","600,995,310",0.001,07/18/2018,"30 N Gould St Ste 12029 Sheridan, WY 82801",smevco.com,+1 615-300-6991,ceo@nitroholdingsllc.com,Unaudited,03/31/2024 Quarterly Report,12/31,"Bryan A Wilkinson (CEO, Acting President), Dou...",NaN,2 as of 01/01/2024
5,SMFKY,"11,552,150,000",NaN,"257,000,000",NaN,,Beech Hill Clonskeagh Dublin 4 Ireland,www.smurfitkappa.com,+353 1 202 7000,ir@smurfitkappa.com,Audited,NaN,12/31,"Gary McGann (CEO), Ian Curley (CFO), Seamus Mu...",NaN,"41,000 as of 09/26/2013"
6,SMFNF,"90,485,443,440",NaN,"1,414,056,000",NaN,NaN,1-2 Yurakucho 1-chome Chiyoda-ku Tokyo 100-000...,www.smbc.co.jp,+81-3-3282-8111,NaN,Audited,03/31/2024,3/31,"Teisuke Kitayama (President), Junji Tanehashi ...",Junji Tanehashi,"23,228 as of 12/31/2013"
7,SMFRF,"7,205,214,874","400,000,000","205,922,117",NaN,NaN,"Shinjuku Green Tower Bldg. 141, Nishi-Shinjuku...",sfc.jp/english,+81-3-5322-6666,NaN,NaN,NaN,12/31,NaN,NaN,NaN
8,SMFTF,"10,561,216,433",NaN,"228,644,774",NaN,,Beech Hill Clonskeagh Dublin Ireland,www.smurfitkappa.com,+353 1 202 7000,ir@smurfitkappa.com,Audited,12/31/2006,12/31,NaN,NaN,NaN
9,SMFX,"922,330","1,100,000,000","922,330",0.00001,,"50 West Liberty Street Suite 880 Reno, NV 89501",www.samtradefx.com,775-322-0626,support@samtradefx.com,Unaudited,08/31/2021 Quarterly Report,2/28,Nai De Goh (CEO),NaN,100 as of 07/28/2021


In [58]:
file = "OTC_Markets_15k_2.csv"

In [59]:
# Converting into the CSV File
df.to_csv(file, index=False)

In [144]:
df1 = pd.read_csv(file)
# df2 = pd.read_csv(file2)
df = pd.concat([df1, df], ignore_index=True)
df

,Company Symbol,Market Cap,Authorized Shares,Outstanding Shares,Par Value,SEC Filings,Address,Website,Phone,Email,Audited Financials,Latest Report,Fiscal Year End,Company Officers,BOARD OF DIRECTORS,Employees
0,AAAIF,NaN,NaN,NaN,NaN,NaN,NaN,www.thealternativeinvestmenttrust.com,61 2 8277 0000,enquiries@oneinvestment.com.au,NaN,NaN,12/31,NaN,NaN,NaN
1,AAALY,2074052673,NaN,59857220,NaN,NaN,Paulinenstrasse 15 Wiesbaden 65189 Germany,www.aareal-bank.com,49-611-348-0,NaN,Audited,NaN,12/31,"Wolf Schumacher (CEO), Jurgen Junginger (Head ...",Wolf Schumacher,NaN
2,AABB,46904183,3600000000,3374401657,0.0001,03/11/2009,3753 Howard Hughes Parkway Suite 200-738 Las V...,www.asiabroadbandinc.com,+1 702-866-9054,ir@asiabroadbandinc.com,Unaudited,03/31/2024 Quarterly Report,12/31,"Chris Torres (President, CEO), Bernard Velez (...","Armando Gonzalez, Luis Roman, Chris Torres, Be...",16 as of 12/31/2023
3,AABVF,2938111,NaN,144877282,NaN,NaN,"65 Queen Street West Suite 815, P.O. Box 75 To...",www.aberdeeninternational.ca,416-861-5875,dstein@aberdeeninternational.ca,Audited,NaN,1/31,"George Faught (CEO), David Stein (President, C...",NaN,14 as of 11/24/2015
4,AACAF,3643270620,NaN,1206381000,NaN,NaN,"AAC Technologies Building,No.18 Xinxi Rd North...",www.aactechnologies.com,86 755 33972018,aac2018@aactechnologies.com,Audited,NaN,12/31,"Benjamin Zhengmin Pan (CEO), Du Kuang Yang (COO)",Koh Boon Hwee,"12,431 as of 12/31/2010"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,CLBN,366,NaN,"36,565,164",0.001,,"541 10th St. NW Atlanta, GA 30318",www.navajowindenergy.com,800-598-2034,NaN,NaN,NaN,12/31,"Barry Doyle (CEO, President), Colin Harris (CFO)","George Brock, Colin Harris",NaN
2996,CLCFF,"3,278,054",Unlimited,"131,122,173",NaN,,"1890 - 1075 West Georgia Street Vancouver, BC ...",www.christinalakecannabis.com,604-687-2038,investors@clcannabis.com,Audited,08/31/2023 Interim Financial Report,11/30,"Mark Aiken (CEO), Nicco Dehaan (COO), Ryan Jam...","James Michael McMillan, Mervin Boychuk, Nicco ...",15 as of 10/31/2023
2997,CLCGY,"4,086,497,500",NaN,"243,970,000",NaN,,Cnr. Searle and Pontac Streets PO Box 5142 Cap...,www.clicksgroup.co.za,+27 21 460 1911,service@clicks.co.za,Audited,NaN,8/31,"David Kneale (CEO), Michael Harvey (Managing D...",NaN,"8,064 as of 08/31/2010"
2998,CLCHY,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
# Checking
df.tail(30)

,Company Symbol,Market Cap,Authorized Shares,Outstanding Shares,Par Value,SEC Filings,Address,Website,Phone,Email,Audited Financials,Latest Report,Fiscal Year End,Company Officers,BOARD OF DIRECTORS,Employees
483,AICOF,"9,324,534",Unlimited,"71,782,402",NaN,NaN,"Unit 1010 ,12471 Horseshoe Way Richmond, BC V7...",www.genai-solutions.com,+1 406-879-7632,info@genai-solutions.com,Audited,01/31/2024 Annual Report,1/31,"Ryan Selby (CEO, Executive Director), Patrick ...","Aaron Bowden, Jordan John Crockett",2 as of 04/19/2023
484,AICXF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
485,AIDA,"5,400",NaN,"27,000,000",0.0010,NaN,31 Dingjiang Road Jianggan District Hangzhou 3...,NaN,86-0571-85802712,NaN,NaN,09/30/2008,12/31,NaN,Jin Biao,NaN
486,AIDG,NaN,"500,000,000","20,000,000",0.0001,12/31/2023,"2-41-7-336, Shinsakae Naka-ku Nagoya-shi Aichi...",81-50-5327-4459,NaN,NaN,Audited,12/31/2023,3/31,NaN,NaN,NaN
487,AIFLF,"734,848,395",NaN,"240,933,900",NaN,NaN,"381-1 Takasago-cho, Gojo-Agaru Karasuma-Dori K...",www.ir-aiful.com/english,075-201-2000,ir@aiful.co.jp,Audited,NaN,3/31,"Yoshitaka Fukuda (President, CEO), Yoshinobu T...",NaN,"1,484 as of 09/30/2012"
488,AIFLY,"679,433,649",NaN,"240,933,918",NaN,NaN,"381-1 Takasago-cho, Gojo-Agaru Karasuma-Dori K...",www.ir-aiful.com/english,075-201-2000,ir@aiful.co.jp,Audited,NaN,3/31,"Yoshitaka Fukuda (President, CEO), Yoshinobu T...",NaN,"1,484 as of 09/30/2012"
489,AIFM,139,"75,000,000","692,997",NaN,NaN,"Unit 503, 5F Silvercord Tower 2 30 Canton Road...",aifarm.co,+86 (852) 91235575,NaN,Audited,01/21/2020 8K Report,12/31,Philip K. H. Chan (CFO),NaN,NaN
490,AIFS,"7,912,428","12,000,000","4,681,910",0.0100,05/17/2004,"15218 Summit Ave Ste 300 Box 240 Fontana, CA 9...",www.auto-graphics.com,+1 909-595-7004,rbs@auto-graphics.com,Audited,03/31/2024 Quarterly Report,12/31,"Paul R. Cope (President), Cheryl Slinkard (COO...","Paul R. Cope, Garry Heath, W. Brent Hicks, Ken...",31 as of 11/01/2023
491,AIGFF,"4,317,951",Unlimited,"62,943,885",NaN,NaN,"750 - 1620 Dickson Ave. Kelowna, BC V1Y 9Y2 Ca...",www.genesisaicorp.com,+1 2508686553,info@genesisaicorp.com,Audited,03/31/2024 Interim Financial Report,6/30,"Devinder Randhawa (CEO), Ryan Cheung (CFO), Mo...","Devinder Randhawa, James Ramsey Bannerman, Jor...",0 as of 06/11/2024
492,AIGI,"2,118","1,500,000,000","21,178,017",0.0010,NaN,"4055 Wilshire Blvd Suite 512 Los Angeles, CA 9...",www.artistmss.com,213-365-1880,artistmssusa@gmail.com,NaN,06/30/2012 Quarterly Report,12/30,"George Kim (President), Sung Jung (Admin. Offi...",NaN,NaN
